In [184]:
import pandas as pd
import numpy as np
import re
import pytz
import os
from pathlib import Path
import sys
sys.path.append("/home/jovyan/shared/service-data")

from src.clean import clean_percentage, normalize_string, standardize_column_names, clean_fiscal_yr
from src.load import load_csv_from_raw
from src.export import export_to_csv
from src.merge import merge_si, merge_ss

base_dir = Path.cwd()
parent_dir = base_dir.parent

In [186]:
si = merge_si()
si.info()

Exported dept.csv to /home/jovyan/shared/service-data/outputs/utils
Exported si.csv to /home/jovyan/shared/service-data/outputs
Exported sid_list.csv to /home/jovyan/shared/service-data/outputs/utils
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8930 entries, 0 to 8929
Data columns (total 52 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   fiscal_yr                                            8930 non-null   object 
 1   service_id                                           8930 non-null   object 
 2   service_name_en                                      8930 non-null   object 
 3   service_name_fr                                      8930 non-null   object 
 4   service_description_en                               8929 non-null   object 
 5   service_description_fr                               8929 non-null   object 
 6   service_type                    

In [199]:
print(si['service_scope'].unique())

si_ext = si.loc[
    (si['service_scope'].str.contains('EXTERN', regex=True)) | 
    (si['service_scope'].str.contains('ENTERPRISE', regex=True))
] 

['ENTERPRISE, EXTERN' 'EXTERN' 'ENTERPRISE, EXTERN, INTERN' 'ENTERPRISE'
 'EXTERN, INTERNCLUSTER' 'EXTERN, INTERN' 'ENTERPRISE, INTERN'
 'ENTERPRISE, EXTERN, INTERN, INTERNCLUSTER' 'ENTERPRISE, INTERNCLUSTER'
 'ENTERPRISE, EXTERN, INTERNCLUSTER' 'EXTERN, INTERN, INTERNCLUSTER'
 'ENTERPRISE, INTERN, INTERNCLUSTER' 'INTERN' 'CLUSTER, EXTERN' 'CLUSTER'
 'CLUSTER, ENTERPRISE, EXTERN, INTERN' 'CLUSTER, EXTERN, INTERN'
 'CLUSTER, ENTERPRISE, INTERN' 'CLUSTER, ENTERPRISE, EXTERN']
<class 'pandas.core.frame.DataFrame'>
Index: 8897 entries, 0 to 8929
Data columns (total 52 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   fiscal_yr                                            8897 non-null   object 
 1   service_id                                           8897 non-null   object 
 2   service_name_en                                      8897 non-null   object 
 3   serv



ifoi_en = pd.read_csv(parent_dir / "inputs" / "ifoi_en.csv")
ifoi_fr = pd.read_csv(parent_dir / "inputs" / "ifoi_fr.csv")

ifoi_en = ifoi_en.set_index(ifoi_en.columns[0], drop=True).add_suffix('_en')
ifoi_fr = ifoi_fr.set_index(ifoi_fr.columns[0], drop=True).add_suffix('_fr')

ifoi_en.set_index(ifoi_en.columns[0], drop=True)
ifoi_fr.set_index(ifoi_fr.columns[0], drop=True)

ifoi = pd.concat([ifoi_en, ifoi_fr], axis=1)

# Extract column lists
en_cols = ifoi_en.columns.tolist()
fr_cols = ifoi_fr.columns.tolist()

# Interleave them by index
merged_cols = [col for pair in zip(en_cols, fr_cols) for col in pair]

# Apply new column order
ifoi = standardize_column_names(ifoi[merged_cols].reset_index())

ifoi